# PROJECT-3. EDA + Feature Engineering. Соревнование на Kaggle. EDA & ML

## 0. Описание кейса и постановка задачи

Представьте, что вы работаете дата-сайентистом в компании Booking. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов обнаружения таких отелей является построение модели, *которая предсказывает рейтинг отеля*. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель ведёт себя нечестно, и его стоит проверить.

Вы будете работать с датасетом, в котором содержатся сведения о 515 000 отзывов на отели Европы. Модель, которую вы будете обучать, должна предсказывать рейтинг отеля по данным сайта Booking на основе имеющихся в датасете данных. Изученные нами навыки разведывательного анализа помогут улучшить модель.

Первоначальная версия датасета содержит 17 полей со следующей информацией:

- hotel_address — адрес отеля;
- review_date — дата, когда рецензент разместил соответствующий отзыв;
- average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
- hotel_name — название отеля;
- reviewer_nationality — страна рецензента;
- negative_review — отрицательный отзыв, который рецензент дал отелю;
- review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
- positive_review — положительный отзыв, который рецензент дал отелю;
- review_total_positive_word_counts — общее количество слов в положительном отзыве.
- reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
- total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
- total_number_of_reviews — общее количество действительных отзывов об отеле;
- tags — теги, которые рецензент дал отелю;
- days_since_review — количество дней между датой проверки и датой очистки;
- additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
- lat — географическая широта отеля;
- lng — географическая долгота отеля.

## 1. Подготовка к работе

### 1.1. Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

# Загружаем специальный инструмент для разбивки данных:  
from sklearn.model_selection import train_test_split

# Импортируем необходимые библиотеки:
# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor
# инструменты для оценки точности модели
from sklearn import metrics

# библиотека для логирования эксперимента
from comet_ml import Experiment

# Библиотеки визуализации
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Установка параметров визуализации
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
sns.set_theme(style='darkgrid')

# Собственный модуль обработки данных
from eda import *

### 1.2. Запуск эксперимента

In [2]:
# Нужно ли логирование в Comet
save_experiment = False

if save_experiment:
    experiment = Experiment(
        api_key='3gDYBYgwtqbTbf3z0fBLFin1U',
        project_name='booking-reviews',
        workspace='mvulf'
    )

### 1.3. Константы обучения

In [3]:
TEST_SIZE = 0.25
RANDOM_STATE = 42
N_ESTIMATORS = 100
TARGET_NAME = 'reviewer_score'

if 'experiment' in locals():
    # Логируем применяемые параметры
    params = {
        'test_size': TEST_SIZE,
        'random_state': RANDOM_STATE,
        'n_estimators': N_ESTIMATORS,
        'target_name': TARGET_NAME
    }
    experiment.log_parameters(params)

### 1.4. Загрузка данных

In [4]:
# Загрузим данные для обучения
hotels = pd.read_csv('input/hotels_train.csv')
hotels.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097


In [5]:
# Загрузим тестовые данные
test = pd.read_csv('input/hotels_test.csv')
test.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443


## 2. Понимание данных

Данный раздел подробно представлен в ноутбуке ["hotels-1-data_understanding.ipynb"](https://github.com/mvulf/sf_data_science/tree/main/project_3/hotels-1-data_understanding.ipynb)

## 3. Подготовка данных

### 3.0 Предподготовка данных

Проводится на основе раздела **2. Понимание данных**

In [6]:
def prepare_data(df):
    """Data preparing function, which would be used for transforming both
    Train and Test

    Args:
        df (pd.DataFrame): DataFrame for transforming

    Returns:
        df (pd.DataFrame): Prepared DataFrame
    """
    # Date conversion
    df['review_date'] = pd.to_datetime(df['review_date'])
    
    # Tags retrieving
    df['splitted_tags'] = df['tags'].str.findall(r"'[\w\s]+'")
    # Tags convertion: delete quotes and out spaces
    df['splitted_tags'] = df['splitted_tags'].\
    apply(lambda x: [y[1:-1].strip() for y in x])
    
    # Convert days_since_review to int
    df['days_since_review'] = df['days_since_review'].\
        apply(lambda x: int(x.split(' ')[0]))
        
    # Retrieve hotel country (consider United Kingdom)
    df['hotel_country'] = df['hotel_address'].apply(
        lambda x: x.split()[-1] if x.split()[-1] != 'Kingdom'\
            else ' '.join(x.split()[-2:])
        )
    
    # Retrieve hotel city (consider United Kingdom)
    df['hotel_city'] = df.apply(
        lambda x: x['hotel_address'].split()[-5] 
            if x['hotel_country'] == 'United Kingdom'\
                else x['hotel_address'].split()[-2], axis=1
        )
    
    return df

print('Предподоготовка обучающей выборки')
prepare_data(hotels).info()
print('Предподоготовка тестовой выборки')
prepare_data(test).info()

Предподоготовка обучающей выборки
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 20 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               386803 non-null  object        
 1   additional_number_of_scoring                386803 non-null  int64         
 2   review_date                                 386803 non-null  datetime64[ns]
 3   average_score                               386803 non-null  float64       
 4   hotel_name                                  386803 non-null  object        
 5   reviewer_nationality                        386803 non-null  object        
 6   negative_review                             386803 non-null  object        
 7   review_total_negative_word_counts           386803 non-null  int64         
 8   total_number_of_reviews                 

**Дополним данные координатами центров городов**

In [7]:
import requests
TOKEN_YA = '8c94603d-6345-41c4-a1b3-e59bb4bb2cb3'
url = 'https://geocode-maps.yandex.ru/1.x'

# Prepare temp column for getting cities and countries
hotels['city_country'] = hotels['hotel_city'] + ', '\
    + hotels['hotel_country']
cities = hotels['city_country'].unique()

# List of cities locations
cities_loc = []

for city in cities:
    # Get response from yandex
    params = {
        'geocode': city,
        'apikey': TOKEN_YA,
        'format': 'json',
        'lang': 'en_US'
    }
    response = requests.get(url, params=params)
    out = response.json()

    # Retrieve location data
    # and object coordinates
    geo_object = out['response']['GeoObjectCollection']\
        ['featureMember'][0]['GeoObject']
    coordinates = geo_object['Point']['pos'].split()
    city_loc = {
        'hotel_city': geo_object['name'],
        'hotel_country': geo_object['metaDataProperty']['GeocoderMetaData']\
            ['AddressDetails']['Country']['CountryName'],
        'center_lng': float(coordinates[0]),
        'center_lat': float(coordinates[1])
        }
    cities_loc.append(city_loc)

# Create df for further merging
cities_loc_df = pd.DataFrame(cities_loc)
display(cities_loc_df)

# Объединим датафреймы
hotels = hotels.merge(cities_loc_df, on=['hotel_city', 'hotel_country'])
hotels.info()

,hotel_city,hotel_country,center_lng,center_lat
0,London,United Kingdom,-0.127696,51.507351
1,Paris,France,2.351556,48.856663
2,Amsterdam,Netherlands,4.892557,52.373057
3,Milan,Italy,9.189595,45.464183
4,Vienna,Austria,16.363460,48.206487
5,Barcelona,Spain,2.144449,41.392696


<class 'pandas.core.frame.DataFrame'>
Int64Index: 386803 entries, 0 to 386802
Data columns (total 23 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               386803 non-null  object        
 1   additional_number_of_scoring                386803 non-null  int64         
 2   review_date                                 386803 non-null  datetime64[ns]
 3   average_score                               386803 non-null  float64       
 4   hotel_name                                  386803 non-null  object        
 5   reviewer_nationality                        386803 non-null  object        
 6   negative_review                             386803 non-null  object        
 7   review_total_negative_word_counts           386803 non-null  int64         
 8   total_number_of_reviews                     386803 non-null  int64        

**Извлечём расстояния от отелей до центров соответствующих городов**

Для этого применим формулу гаверсинусов

$$\Delta l = 2r\cdot arcsin(\sqrt{hav(\phi_2 - \phi_1) + cos(\phi_1)cos(\phi_2)hav(\lambda_2 - \lambda_1)})$$

где $hav(\theta) = sin^2(\theta/2)$; $\phi_1$, $\phi_2$ - широты точек; 
$\lambda_1$, $\lambda_2$ - долготы точек в радианах

Проверим корректность формулы

In [8]:
# Проверка валидности по расстоянию между Лондоном и Парижем
london_paris = hav_distance(51.507351, -0.127696, 48.856663, 2.351556)
print(f'Лондон-Париж: {london_paris} km')

Лондон-Париж: 343.5180236172188 km


Считает с точностью до километра.
Погрешность вызвана осреднением радиуса Земли.

Вычислим расстояния от центров соответствующих городов до отелей

In [16]:
hotels['center_distance'] = hotels.apply(lambda row:\
    hav_distance(row['center_lat'], row['center_lng'],\
        row['lat'], row['lng']), axis=1)

hotels['center_distance'].describe()

count    384355.000000
mean          3.238309
std           2.529124
min           0.072794
25%           1.506401
50%           2.594047
75%           4.091961
max          17.145189
Name: center_distance, dtype: float64

Медианное расстояние от центра города до гостиницы - 2.59 км. 
Заполним пропуски этим расстоянием в разделе 3.1

In [17]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 386803 entries, 0 to 386802
Data columns (total 24 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               386803 non-null  object        
 1   additional_number_of_scoring                386803 non-null  int64         
 2   review_date                                 386803 non-null  datetime64[ns]
 3   average_score                               386803 non-null  float64       
 4   hotel_name                                  386803 non-null  object        
 5   reviewer_nationality                        386803 non-null  object        
 6   negative_review                             386803 non-null  object        
 7   review_total_negative_word_counts           386803 non-null  int64         
 8   total_number_of_reviews                     386803 non-null  int64        

### 3.1 Очистка данных
Проводится: 
- удаление/разметка дубликатов;
- заполнение пропусков (при необходимости);
- удаление/разметка выбросов и аномалий

#### Дубликаты

In [41]:
columns_for_duplicates = list(hotels.columns)
columns_for_duplicates.remove('splitted_tags')
dup_count = hotels.duplicated(columns_for_duplicates).sum()
print('Количество дубликатов:', dup_count)
print('Размер датафрейма до удаления дубликатов:', hotels.shape[0])
hotels = hotels.drop_duplicates(columns_for_duplicates)
print('Дубликаты удалены. Размер датафрейма:', hotels.shape[0])

Количество дубликатов: 307
Размер датафрейма до удаления дубликатов: 386803
Дубликаты удалены. Размер датафрейма: 386496


#### Пропуски

- Предварительный анализ данных показал, что явные пропуски содержатся только в **lat** и **lng**
- Неявные пропуски имеются в **positive_review** и **negative_review**

##### **LAT** и **LNG**

In [ ]:
# заполним пропуски медианным расстоянием от центров городов

#### Выбросы и аномалии

- Необходимо сделать разные имена для Отеля Регина из разных стран
- Можно отметить выбросы в колонке additional_number_of_scoring, или же перейти к относительному количеству оценок без отзывов
- Выбросы в остальных колонках убираются засчёт введения категориального признака

### 3.2 Разведывательный анализ данных (EDA)

Как правило, **включает в себя**:
- Проектирование признаков (Feature Engineering);
- Кодирование признаков (Ordered-, OneHot-, Binary- encoding);
- Проверка статистических гипотез;
- Отбор признаков (Feature Selection)

### Отбор признаков
Для категориальных - хи-квадрат

Для непрерывных - ANOVA

### Решение-заглушка:

In [6]:
# убираем признаки, которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, 
# просто выберем их и удалим
object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
hotels.drop(object_columns, axis = 1, inplace=True)

# заполняем пропуски самым простым способом
hotels = hotels.fillna(0)

## 4. ML-Моделирование

**ОСНОВНЫЕ ЭТАПЫ:**
1. **Разделение набора данных:**
    - "по столбцам":
        - Признаки для обучения модели - **X**
        - Целевая переменная, которую будем предсказывать - **y**
    - "по строкам":
        - Тренировочный набор (**train**), для обучения модели
        - Тестовый набор (**test**), для оценки точности модели
2. **Создание; обучение модели и предсказание значений**
3. **Оценка качества модели**
    - с помощью метрик проверяется точность прогнозов, сделанных моделью.

### 4.1. Разделение набора данных

In [7]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х — данные с информацией об отелях, у — целевая переменная (рейтинги отелей)  
X = hotels.drop([TARGET_NAME], axis=1)
y = hotels[TARGET_NAME]

# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" — для тестирования.
# Для тестирования мы будем использовать 25 % от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

### 4.2. Создание и обучение модели

Примечание: 
- для корректной работы все данные в датафрейме, который используется при обучении модели, должны быть в числовых форматах int или float;
- в столбцах не должно быть пропущенных значений. Вместо каждого пропущенного значения вам нужно будет вычислить и поместить в ячейку максимально близкое к реальности значение.

In [8]:
# Создаём модель 
regr = RandomForestRegressor(n_estimators=N_ESTIMATORS)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred 
y_pred = regr.predict(X_test)

### 4.3. Оценка качества модели

In [9]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
train_mape = metrics.mean_absolute_percentage_error(y_test, y_pred)
print('MAPE:', train_mape)
# Запишем показатели метрики
metrics = {
    'train_mape': train_mape
}
experiment.log_metrics(metrics)

MAPE: 0.14137276548008906


### Завершение эксперимента

In [18]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/mvulf/booking-reviews/ad4ee937797e427b887b884803a7bd7c
COMET INFO:   Parameters:
COMET INFO:     n_estimators : 100
COMET INFO:     random_state : 42
COMET INFO:     target_name  : reviewer_score
COMET INFO:     test_size    : 0.25
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specification          : 1
COMET INFO:     environment details          : 1
COMET INFO:     filename                     : 1
COMET INFO:     git metadata                 : 1
COMET INFO:     git-patch (uncompressed)     : 1 (4.98 KB)
COMET INFO:     installed packages           : 1
COMET INFO:     notebook                     : 1
COMET INFO:     source_code                  : 1
C

## *5. Валидация*
Должна проводиться проверка того, что построенная модель решает поставленные бизнес задачи; что нет проблем бизнеса, которые не были рассмотрены.

На вход поступает уже построенная модель с высокой используемой метрикой, полученной на шаге моделирования.